# Contents:

#### Import libraries and establish data frame objects
#### Fixing an outlier in the prices column
#### Grouping and aggregating a subset of the dataframe
#### Creating a loyalty flag for existing customers
#### Creating a spending flag
#### Creating an order frequency flag
#### Crosstabulations
#### Exporting and saving the dataframe

# Import libraries and establish data frame objects

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path=r'C:\Users\Shelb\OneDrive\Documents\CF Coursework\Python Fundamentals for Data Analysts\10-2022 Instacart Basket Analysis'

In [3]:
df=pd.read_pickle(os.path.join(path,'Data','Prepared Data','11-9_orders_products_merged.pkl'))

In [4]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0_x', 'order_id', 'user_id', 'order_number',
       'order_day_of_week', 'hour_order_placed', 'days_since_prior_order',
       'new_customer', 'product_id', 'add_to_cart_order', 'reordered',
       'Unnamed: 0_y', 'product_name', 'aisle_id', 'department_id', 'prices',
       '_merge', 'price_range', 'busiest_day', 'busiest_days',
       'busiest_period_of_day'],
      dtype='object')

In [5]:
# To create a dataframe without unnamed, _merge, and busiest_day (as opposed to busiest_days) columns 
df_trim=df[['order_id', 'user_id', 'order_number',
       'order_day_of_week', 'hour_order_placed', 'days_since_prior_order',
       'new_customer', 'product_id', 'add_to_cart_order', 'reordered', 'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range', 'busiest_days',
       'busiest_period_of_day']]

## Fixing an outlier in the prices column

In [6]:
# To check for items which cost upward of 100 dollars
df_trim.loc[df_trim['prices']>100]

,order_id,user_id,order_number,order_day_of_week,hour_order_placed,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day
10030345,912404,17,12,2,14,5.0,False,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range product,Regularly Busy,Most Orders
10030346,603376,17,22,6,16,4.0,False,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range product,Regularly Busy,Most Orders
10030347,3264360,135,2,2,21,13.0,False,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range product,Regularly Busy,Average Orders
10030348,892534,135,3,0,8,12.0,False,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range product,Busiest Day,Average Orders
10030349,229704,342,8,1,19,30.0,False,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range product,Busiest Day,Average Orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29166209,2249946,204099,29,0,8,4.0,False,33664,1,0,2 % Reduced Fat Milk,84,16,99999.0,High range product,Busiest Day,Average Orders
29166210,2363282,204099,31,0,9,2.0,False,33664,1,1,2 % Reduced Fat Milk,84,16,99999.0,High range product,Busiest Day,Most Orders
29166211,3181945,204395,13,3,15,8.0,False,33664,25,0,2 % Reduced Fat Milk,84,16,99999.0,High range product,Least Busy,Most Orders
29166212,2486215,205227,7,3,20,4.0,False,33664,8,0,2 % Reduced Fat Milk,84,16,99999.0,High range product,Least Busy,Average Orders


In [7]:
# To mark outliers as NaN to remove them from future calculations
df_trim.loc[df_trim['prices']>100,'prices']=np.nan

In [8]:
df_trim['prices'].max()

25.0

# Grouping and aggregating a subset of the dataframe

## Subset aggregations

In [9]:
# To create a subset containing the first 1,000,000 rows of df_trim
df_sub=df_trim[:1000000]

In [10]:
df_sub.shape

(1000000, 17)

In [11]:
df_sub.head()

,order_id,user_id,order_number,order_day_of_week,hour_order_placed,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid range product,Regularly Busy,Average Orders
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Average Orders
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Most Orders
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Average Orders
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Most Orders


In [12]:
# To group by the product_name column
df_sub.groupby('product_name')

In [13]:
# To aggregate to produce a single descriptive statistic for the order_number column and group by department_id
df_sub.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [14]:
# To produce multiple descriptive statistics for the order_number column and group by department_id
df_sub.groupby('department_id').agg({'order_number':['mean','min','max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

## Whole set aggregations

In [15]:
# To find the aggregate mean for the order_number column for the whole set, grouped by department_id
df_trim.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


### The subset mean of the order_number column for department 16 is quite a bit higher than the whole set mean for that department, and for department 17 the subset mean of the order_number column is quite a bit lower than the whole set mean

# Creating a loyalty flag for existing customers

In [16]:
# To transform the order_number column to generate the maximum orders for each user
df_trim['max_order']=df_trim.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Shelb\AppData\Local\Temp\ipykernel_4332\924742914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim['max_order']=df_trim.groupby(['user_id'])['order_number'].transform(np.max)


In [17]:
df_trim.head(15)

,order_id,user_id,order_number,order_day_of_week,hour_order_placed,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid range product,Regularly Busy,Average Orders,10
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Average Orders,10
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Most Orders,10
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Average Orders,10
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Most Orders,10
5,3367565,1,6,2,7,19.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Regularly Busy,Average Orders,10
6,550135,1,7,1,9,20.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Busiest Day,Most Orders,10
7,3108588,1,8,1,14,14.0,False,196,2,1,Soda,77,7,9.0,Mid range product,Busiest Day,Most Orders,10
8,2295261,1,9,1,16,0.0,False,196,4,1,Soda,77,7,9.0,Mid range product,Busiest Day,Most Orders,10
9,2550362,1,10,4,8,30.0,False,196,1,1,Soda,77,7,9.0,Mid range product,Least Busy,Average Orders,10


In [18]:
# To create a loyalty flag for each customer
df_trim.loc[df_trim['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_trim.loc[(df_trim['max_order'] <= 40) & (df_trim['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_trim.loc[df_trim['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

C:\Users\Shelb\AppData\Local\Temp\ipykernel_4332\1411585422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[df_trim['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [19]:
df_trim['loyalty_flag'].value_counts(dropna=False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [20]:
df_trim[['user_id','loyalty_flag','order_number']].head(60)

,user_id,loyalty_flag,order_number
0,1,New customer,1
1,1,New customer,2
2,1,New customer,3
3,1,New customer,4
4,1,New customer,5
5,1,New customer,6
6,1,New customer,7
7,1,New customer,8
8,1,New customer,9
9,1,New customer,10


# Creating a spending flag

In [21]:
# To group the average price of goods purchased by loyalty rating
df_trim.groupby('loyalty_flag').agg({'prices':['mean']})

,prices
,mean
loyalty_flag,
Loyal customer,7.773575
New customer,7.801206
Regular customer,7.798262


### There appears to be an inverse relationship between customer loyalty and the average price spent by each loyalty group

In [22]:
# To create the spending flag
df_trim.loc[df_trim['prices']>=10,'spending_flag']='High spender'
df_trim.loc[df_trim['prices']<10,'spending_flag']='Low spender'

C:\Users\Shelb\AppData\Local\Temp\ipykernel_4332\2043308914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim.loc[df_trim['prices']>=10,'spending_flag']='High spender'


In [23]:
df_trim['spending_flag'].value_counts(dropna=False)

Low spender     21760937
High spender    10638795
NaN                 5127
Name: spending_flag, dtype: int64

In [24]:
df_trim[['user_id','prices','spending_flag','loyalty_flag']].head(15)

,user_id,prices,spending_flag,loyalty_flag
0,1,9.0,Low spender,New customer
1,1,9.0,Low spender,New customer
2,1,9.0,Low spender,New customer
3,1,9.0,Low spender,New customer
4,1,9.0,Low spender,New customer
5,1,9.0,Low spender,New customer
6,1,9.0,Low spender,New customer
7,1,9.0,Low spender,New customer
8,1,9.0,Low spender,New customer
9,1,9.0,Low spender,New customer


# Creating an order frequency flag

In [25]:
# To create a column with the aggreagate median of days_since_prior_order by user_id
df_trim['avg_days_since_last_order']=df_trim.groupby('user_id')['days_since_prior_order'].transform(np.median)

C:\Users\Shelb\AppData\Local\Temp\ipykernel_4332\3757540451.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trim['avg_days_since_last_order']=df_trim.groupby('user_id')['days_since_prior_order'].transform(np.median)


In [26]:
# To create the order_frequency flag
df_trim.loc[df_trim['avg_days_since_last_order']>20,'order_frequency']='Non-frequent customer'
df_trim.loc[(df_trim['avg_days_since_last_order']>10)&(df_trim['avg_days_since_last_order']<=20),'order_frequency']='Regular customer'
df_trim.loc[df_trim['avg_days_since_last_order']<=10,'order_frequency']='Frequent customer'

In [27]:
df_trim['order_frequency'].value_counts(dropna=False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency, dtype: int64

In [28]:
df_trim.head(20)

,order_id,user_id,order_number,order_day_of_week,hour_order_placed,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,...,department_id,prices,price_range,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending_flag,avg_days_since_last_order,order_frequency
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,Mid range product,Regularly Busy,Average Orders,10,New customer,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,7,9.0,Mid range product,Least Busy,Average Orders,10,New customer,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,7,9.0,Mid range product,Least Busy,Most Orders,10,New customer,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,7,9.0,Mid range product,Least Busy,Average Orders,10,New customer,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,7,9.0,Mid range product,Least Busy,Most Orders,10,New customer,Low spender,20.5,Non-frequent customer
5,3367565,1,6,2,7,19.0,False,196,1,1,...,7,9.0,Mid range product,Regularly Busy,Average Orders,10,New customer,Low spender,20.5,Non-frequent customer
6,550135,1,7,1,9,20.0,False,196,1,1,...,7,9.0,Mid range product,Busiest Day,Most Orders,10,New customer,Low spender,20.5,Non-frequent customer
7,3108588,1,8,1,14,14.0,False,196,2,1,...,7,9.0,Mid range product,Busiest Day,Most Orders,10,New customer,Low spender,20.5,Non-frequent customer
8,2295261,1,9,1,16,0.0,False,196,4,1,...,7,9.0,Mid range product,Busiest Day,Most Orders,10,New customer,Low spender,20.5,Non-frequent customer
9,2550362,1,10,4,8,30.0,False,196,1,1,...,7,9.0,Mid range product,Least Busy,Average Orders,10,New customer,Low spender,20.5,Non-frequent customer


## Crosstabulations

In [31]:
# To crosstabulate loyalty_flag with spending_flag
crosstab_loyalty_spending=pd.crosstab(df_trim['loyalty_flag'],df_trim['spending_flag'],dropna=False)

In [32]:
crosstab_loyalty_spending

spending_flag,High spender,Low spender
loyalty_flag,,
Loyal customer,3351872,6930891
New customer,2062155,4180686
Regular customer,5224768,10649360


In [33]:
crosstab_loyalty_spending.to_clipboard()

In [34]:
# To crosstabulate loyalty_flag with order_frequency
crosstab_loyalty_frequency=pd.crosstab(df_trim['loyalty_flag'],df_trim['order_frequency'],dropna=False)

In [35]:
crosstab_loyalty_frequency

order_frequency,Frequent customer,Non-frequent customer,Regular customer
loyalty_flag,,,
Loyal customer,10279960,0,4133
New customer,1648671,2724962,1870352
Regular customer,9631222,911475,5334079


In [36]:
crosstab_loyalty_frequency.to_clipboard()

# Exporting the dataframe

In [29]:
df_trim.to_pickle(os.path.join(path,'Data','Prepared Data','11-13_orders_products_merged.pkl'))